<a href="https://colab.research.google.com/github/Lionel98/CTI/blob/main/Test_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install datasets

In [2]:
pip install --upgrade huggingface_hub

In [3]:
pip install transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import pandas as pd
from transformers import pipeline

def roberta_test(csv_file):
    # Load the pipeline
    pipe = pipeline(model="ltkw98/Roberta4")

    # Read the existing CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    #Total Cells
    total_rows=len(df)

    # Iterate over the sentences and update the columns
    for i in range(total_rows):
        # Update the "Model's Response" column
        result = pipe(df['sentence'][i], top_k=1, truncation=True)
        predicted_label = result[0]['label']
        df.loc[i, "Model's Response"] = predicted_label

        # Update the "Log Probability" column
        result = pipe(df['sentence'][i], top_k=5, truncation=True)
        log_probs = [str(entry) for entry in result]
        df.loc[i, "Log Probability"] = ', '.join(log_probs)

    # Save the DataFrame to a new CSV file
    new_csv_file = "roberta_result_set.csv"
    df.to_csv(new_csv_file, index=False)

In [5]:
def bert_test(csv_file):
    # Load the pipeline
    pipe = pipeline(model="ltkw98/Bert1")

    # Read the existing CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    #Total Cells
    total_rows=len(df)

    # Iterate over the sentences and update the columns
    for i in range(total_rows):
        # Update the "Model's Response" column
        result = pipe(df['sentence'][i], top_k=1, truncation=True)
        predicted_label = result[0]['label']
        df.loc[i, "Model's Response"] = predicted_label

        # Update the "Log Probability" column
        result = pipe(df['sentence'][i], top_k=5, truncation=True)
        log_probs = [str(entry) for entry in result]
        df.loc[i, "Log Probability"] = ', '.join(log_probs)

    # Save the DataFrame to a new CSV file
    new_csv_file = "bert_result_set.csv"
    df.to_csv(new_csv_file, index=False)

In [6]:
def minilm_test(csv_file):
    # Load the pipeline
    pipe = pipeline(model="ltkw98/Minilm3")

    # Read the existing CSV file into a DataFrame
    df = pd.read_csv(csv_file)

    #Total Cells
    total_rows=len(df)

    # Iterate over the sentences and update the columns
    for i in range(total_rows):
        # Update the "Model's Response" column
        result = pipe(df['sentence'][i], top_k=1, truncation=True,max_length=512)
        predicted_label = result[0]['label']
        df.loc[i, "Model's Response"] = predicted_label

        # Update the "Log Probability" column
        result = pipe(df['sentence'][i], top_k=5, truncation=True,max_length=512)
        log_probs = [str(entry) for entry in result]
        df.loc[i, "Log Probability"] = ', '.join(log_probs)

    # Save the DataFrame to a new CSV file
    new_csv_file = "minilm_result_set.csv"
    df.to_csv(new_csv_file, index=False)

In [7]:
from datasets import load_dataset

def model_response(csv_file):
    # Load the dataset
    test = load_dataset("csv", data_files=csv_file)

    total_rows = len(test['train']["Model's Response"])
    matching_count = 0

    for i in range(total_rows):
        if test['train']["Model's Response"][i] == test['train']['tec_name'][i]:
            matching_count += 1

    matching_percentage = (matching_count / total_rows) * 100

    print(f"Number of Sentences Matched: {matching_count}")
    print(f"Matching Percentage: {matching_percentage}%")

    return matching_count, matching_percentage

In [8]:
from datasets import load_dataset
import re

def Top5_Response(csv_file):
    # Load the dataset
    test = load_dataset("csv", data_files=csv_file)

    total_rows = len(test['train']['tec_name'])
    match_count = 0

    for i in range(total_rows):
        tec_name = test['train']['tec_name'][i]
        log_probabilities_str = test['train']['Log Probability'][i]
        log_probabilities = re.findall(r"'label': '([^']+)'", log_probabilities_str)

        for label in log_probabilities:
            if tec_name.lower() in label.lower():
                match_count += 1
                break  # Break out of the inner loop once a match is found

    match_percentage = (match_count / total_rows) * 100

    print(f"Number of Matches: {match_count}")
    print(f"Matching Percentage: {match_percentage}%")

    return match_count, match_percentage

In [9]:
#TEST
from datasets import load_dataset
import pandas as pd
from transformers import pipeline

pipe = pipeline(model="ltkw98/Minilm3")
csv_file = "test_result.csv"
test = load_dataset("csv", data_files=csv_file)

total_rows = len(test['train']["Model's Response"])

for i in range(10):
  print(i)
  print(test['train']["sentence"][i])
  result=pipe(test['train']["sentence"][i], top_k=1, truncation=True)
  print(result)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


  0%|          | 0/1 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


0
FIN7 sent spearphishing emails with either malicious Microsoft Documents or RTF files attached.
[{'label': 'Spearphishing Attachment', 'score': 0.16495268046855927}]
1
Winnti for Linux has the ability to deploy modules directly from command and control  servers, possibly for remote command execution, file exfiltration, and socks5 proxying on the infected host.
[{'label': 'Remote Access Software', 'score': 0.19399431347846985}]
2
APT29 has acquired C2 domains, sometimes through resellers.
[{'label': 'Domains', 'score': 0.4050115942955017}]
3
Goopy has the ability to side-load malicious DLLs with legitimate applications from Kaspersky, Microsoft, and Google.
[{'label': 'DLL Side-Loading', 'score': 0.3453075587749481}]
4
NETWIRE has been spread via e-mail campaigns utilizing malicious attachments.
[{'label': 'Malicious Link', 'score': 0.16711021959781647}]
5
As previously mentioned, the samples we obtained range from 494.7 MB to as large as 929.7 MB in size.
[{'label': 'INVALID', 'score

In [10]:
csv_file = "test_result.csv"
roberta_test(csv_file)

In [11]:
bert_test(csv_file)

In [12]:
minilm_test(csv_file)

### ROBERTA RESULTS:

In [13]:
roberta_result = "/content/roberta_result_set.csv"
matching_count, matching_percentage = model_response(roberta_result)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a5561d86d174d6d7/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Number of Sentences Matched: 1825
Matching Percentage: 77.3960983884648%


In [14]:
matching_count, matching_percentage = Top5_Response(roberta_result)

  0%|          | 0/1 [00:00<?, ?it/s]

Number of Matches: 2140
Matching Percentage: 90.754877014419%


### BERT RESULTS:

In [15]:
bert_result = "/content/bert_result_set.csv"
matching_count, matching_percentage = model_response(bert_result)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c1355152cc4b7a7c/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Number of Sentences Matched: 1820
Matching Percentage: 77.18405428329092%


In [16]:
matching_count, matching_percentage = Top5_Response(bert_result)

  0%|          | 0/1 [00:00<?, ?it/s]

Number of Matches: 2109
Matching Percentage: 89.44020356234097%


### MINILM RESULTS:

In [17]:
minilm_result = "/content/minilm_result_set.csv"
matching_count, matching_percentage = model_response(minilm_result)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-dd55ed5df8376983/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Number of Sentences Matched: 1512
Matching Percentage: 64.12213740458014%


In [18]:
matching_count, matching_percentage = Top5_Response(minilm_result)

  0%|          | 0/1 [00:00<?, ?it/s]

Number of Matches: 1989
Matching Percentage: 84.35114503816794%
